In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Input, Reshape, Lambda, Concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Permute
from keras.layers import Conv3D, Deconvolution3D, MaxPooling3D, UpSampling3D, ZeroPadding3D

- Define the loss function and verify it

In [ ]:
def RadiationLoss(y_true, y_pred):
    """
    Compute the loss for the radiation matrix.
    
    Inputs:
    - y_true: radiation of the target building. 3D Tensor with radiation value at taget surface and others 0.
    - y_pred: the prediction of the radiation.
    
    Returns:
    - scalar mse loss, only calculated where radiation value not equal to zero
    """
    
    y_loc = K.cast(K.not_equal(y_true,K.constant(0)),'float')
    return K.sum(K.pow(y_true-y_pred*y_loc,2))/K.sum(y_loc)
    

In [ ]:
radiation = np.random.randn(10,10,10,1)
radiation[:5,:5,:5,:] = 0
prediction = np.random.randn(10,10,10,1)
a = K.constant(radiation)
b = K.constant(prediction)
loss = RadiationLoss(a, b)
print('Result:          ', K.eval(loss))
print('Expected result: ', np.sum((radiation-prediction*(radiation!=0))**2)/np.sum(radiation!=0))

- Define the network with Voxnet structure

In [ ]:
matrix_size = (50, 50, 50, 1)

inp = Input(matrix_size)

# Voxnet structure + autoencoder
enc = Conv3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(inp)
enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(enc)
enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)
conv_shape = enc.get_shape().as_list()

enc = Flatten()(enc)
latent = Dense(256, activation='relu')(enc)

dec = Dense(np.prod(conv_shape[1:]), activation='relu')(latent)
dec = Reshape(conv_shape[1:])(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = Deconvolution3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Deconvolution3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = ZeroPadding3D(((0,1),(0,1),(0,1)))(dec) # pad 0s at one side to match the size
dec = Deconvolution3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Deconvolution3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(dec)

out = Conv3D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(dec) # Assume normalized data [0,1]

voxnet_model = Model(inp, out)
voxnet_model.compile(optimizer='adam',loss=RadiationLoss)

In [ ]:
voxnet_model.summary()

In [ ]:
X = np.random.rand(128, 50, 50, 50, 1)
y = np.random.rand(128, 50, 50, 50, 1)
y[:,:25,:,:,:] = 0

In [ ]:
history = voxnet_model.fit(X, y, epochs=1)

- Check loss again

In [ ]:
y_pred = voxnet_model.predict(X[:5])
print('Result:          ', voxnet_model.test_on_batch(X[:5],y[:5]))
print('Expected result: ', np.sum((y[:5]-y_pred[:5]*(y[:5]!=0))**2)/np.sum(y[:5]!=0))

- Define the network with 2D-3D structure

In [ ]:
def extractChannel(inp, i):
    # Input 2D image with K channel. extract i-th channel
    return K.expand_dims(inp[:,:, i])

def sqeezeChannel(inp):
    return K.squeeze(inp,axis=-1)

def expandChannel(inp):
    return K.expand_dims(inp)

In [ ]:
matrix_size = (50, 50, 50, 1)
last_channel = 30

# 2D network
def sub2Dnetwork_1c():
    inp_2D = Input((matrix_size[0], matrix_size[1], 1))

    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((4,4))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = UpSampling2D((4,4))(x)
    x = ZeroPadding2D(1)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(last_channel, padding='same', kernel_size=3)(x)

    return  Model(inp_2D, out)

def sub2Dnetwork_50c():
    inp_2D = Input((matrix_size[:-1]))

    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((4,4))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = UpSampling2D((4,4))(x)
    x = ZeroPadding2D(1)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(last_channel, padding='same', kernel_size=3)(x)

    return  Model(inp_2D, out)

def combine2Dnetwork():
    inp_2D = Input((matrix_size[0], matrix_size[1], last_channel*(matrix_size[2]+1)))
    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, padding='same', kernel_size=3)(x)
    x = UpSampling2D((2,2))(x)
    x = ZeroPadding2D(((0,1),(0,1)))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(matrix_size[2], padding='same', kernel_size=3)(x)

    return Model(inp_2D, out)

def get2Dnetwork():
    inp = Input(matrix_size[:-1])
    subnetworks = []
    suboutputs = []
    for i in range(matrix_size[-2]):
        subnet = sub2Dnetwork_1c()
        subout = subnet(Lambda(lambda x: extractChannel(x, i))(inp))
        suboutputs.append(subout)
        subnetworks.append(subnet)
    subnet = sub2Dnetwork_50c()
    subout = subnet(inp)
    suboutputs.append(subout)
    subnetworks.append(subnet)

    inp_combined = Concatenate()(suboutputs)
    comb = combine2Dnetwork()
    return Model(inp,comb(inp_combined))

In [ ]:
# 3D network
inp = Input(matrix_size)
inp_2D = Lambda(sqeezeChannel)(inp)

X_model = get2Dnetwork()
Y_model = get2Dnetwork()
Z_model = get2Dnetwork()

inp_2D_X = Permute((1,2,3))(inp_2D)
inp_2D_Y = Permute((1,3,2))(inp_2D)
inp_2D_Z = Permute((2,3,1))(inp_2D)

out_2D_X = X_model(inp_2D_X)
out_2D_Y = Y_model(inp_2D_Y)
out_2D_Z = Z_model(inp_2D_Z)

inp_3D = Concatenate()([inp,
                        Lambda(expandChannel)(out_2D_X),
                        Lambda(expandChannel)(out_2D_Y),
                        Lambda(expandChannel)(out_2D_Z)])

enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inp_3D)
enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(128, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = Conv3D(128, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
dec = UpSampling3D((2,2,2))(enc)
dec = ZeroPadding3D(((0,1),(0,1),(0,1)))(dec) # pad 0s at one side to match the size

dec = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = UpSampling3D((2,2,2))(dec)

dec = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)

out = Conv3D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(dec) # Assume normalized data [0,1]

lr2D_model = Model(inp, out)
lr2D_model.compile(optimizer='adam',loss=RadiationLoss)

In [ ]:
lr2D_model.summary()

In [ ]:
history = lr2D_model.fit(X, y, epochs=1)